Preparação para o Active Learning

Será considerado o cenário simulado onde temos um pool de $5.240$ lesões, mas apenas uma pequena parte delas está rotulada com o diagnóstico final.

Separação do Pool Inicial Rotulado: Início com apenas $5\%$ ($262$ lesões) aleatórias do seu df_final como seu conjunto Initial Labeled Set ($L_0$).

Pool Não Rotulado: O restante ($4.978$ lesões) é o Unlabeled Pool ($U$).

Próximos passos:

Desenvolvimento do Modelo: Defina a arquitetura da CNN (usando as imagens) e um método para incorporar os dados MONET_... (por exemplo, como side information ou como um passo inicial de pré-classificação).

Estratégia de Aquisição (O cerne do Active Learning): Defina como você usará os dados MONET_... para selecionar as próximas $X$ amostras do pool $U$ a serem rotuladas.

## Adaptação para treino ASSL
#### 1 - Separação em conjuntos L (rotulado) e U (não rotulado)
- L: contém imagens com rótulos verdadeiros (serão usadas na perda supervisionada).

- U: contém imagens sem rótulos (usadas na perda não supervisionada e no cálculo das métricas por amostra).
Durante o treinamento ativo, novas amostras de U podem ser movidas para L conforme forem selecionadas (Top-K com maior Score).

#### 2 - Augmentations: aumentos, incrementos ou ampliações de modo geral. já temos um tratamento realizado para utilizar as imagens do dataset no Keras. O artigo cita dois tipos de transformações:

O objetivo é gerar versões diferentes da mesma imagem para medir consistência.

Weak augmentation: transformações leves (flip, crop, normalize). Mantém a semântica da imagem.
→ Usada para gerar o pseudo-rótulo confiável.

Strong augmentation: transformações mais intensas (RandAugment ou CTAugment).
→ Usada para verificar se o modelo é consistente ao prever a mesma classe mesmo sob distorções.

Essas duas versões da imagem são processadas juntas em FixMatch.

#### FixMatch
Método de aprendizado semi-supervisionado que combina consistência e pseudo-rotulagem:

O modelo gera uma predição para a imagem weak.

Se a probabilidade máxima for ≥ τ (ex.: 0.95), essa classe vira o pseudo-rótulo.

A imagem strong é usada para calcular uma perda não supervisionada (cross-entropy entre o pseudo-rótulo e a predição da versão strong).
Isso permite treinar com imagens não rotuladas usando apenas as predições mais confiantes.

Obs: As Random layers* do Keras se aproximam da ideia do FixMatch

3 - Estruturas para EMA + UCB
EMA (Exponential Moving Average)

Técnica que mantém uma média suavizada ao longo do tempo.
Serve para reduzir flutuações bruscas na incerteza de uma amostra.
Fórmula:




onde α é a taxa de atualização (geralmente 0.8).

UCB (Upper Confidence Bound)

Usada para ponderar média + variância de uma métrica, originada do reinforcement learning.
Ela favorece amostras com alta incerteza ou alta variação (potencialmente informativas).


	​


onde c controla o grau de exploração (típico: 0.5 para incerteza e 2.0 para inconsistência).

Essas estruturas são aplicadas tanto à incerteza quanto à inconsistência de cada amostra U.
Precisamos de um ID por amostra em U para acumular EMA/UCB corretamente. Se seu train_dataset já é tf.data.Dataset

#### 4 - Definir métricas por amostra

Pseudo-EL2N: 

KL simétrica:

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers
from config import H, W, NUM_CLASSES

# as imagens são definidas como image_size=(180, 180) na divisão do dataset pro tensorflow

INPUT_SHAPE = (H, W, 3)  # defina H,W conforme seu dataset

def build_model():
    model = keras.Sequential([
        layers.Input(shape=INPUT_SHAPE),
        layers.Rescaling(1./255),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.Flatten(),
        layers.Dense(NUM_CLASSES)  # logits
    ])
    return model

model = build_model()
optimizer = keras.optimizers.Adam()
